# 패키지 로드

In [1]:
%pip install torch datasets
%pip install transformers accelerate peft trl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

/Users/jungikim/miniconda3/envs/skn-final/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 데이터 분석 및 전처리

In [3]:
# 허깅페이스 허브에서 데이터셋 로드
dataset = load_dataset("iamjoon/klue-mrc-ko-rag-dataset", split="train")

# 원본 데이터의 type별 분포 출력
print("원본 데이터의 type 분포:")
for type_name in set(dataset['type']):
    print(f"{type_name}: {dataset['type'].count(type_name)}")

Generating train split: 100%|██████████| 1884/1884 [00:00<00:00, 20812.61 examples/s]

원본 데이터의 type 분포:
paraphrased_question: 196
mrc_question: 491
no_answer: 404
synthetic_question: 497
mrc_question_with_1_to_4_negative: 296


## train/test 데이터 분할

> 데이터의 타입 별로 train/test 테이터셋 비율을 산정한다.

In [4]:
# train/test 분할 비율 설정 (0.5면 5:5로 분할)
test_ratio = 0.8

train_data = []
test_data = []

# type별로 순회하면서 train/test 데이터 분할
for type_name in set(dataset['type']):
    # 현재 type에 해당하는 데이터의 인덱스만 추출
    curr_type_data = [i for i in range(len(dataset)) if dataset[i]['type'] == type_name]

    # test_ratio에 따라 test 데이터 개수 계산
    test_size = int(len(curr_type_data) * test_ratio)

    # 현재 type의 데이터를 test_ratio 비율로 분할하여 추가
    test_data.extend(curr_type_data[:test_size])
    train_data.extend(curr_type_data[test_size:])

## 데이터를 OpenAI format으로 가공

In [5]:
# system_message 정의
system_message = """당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
{search_result}"""


# OpenAI format으로 데이터 변환을 위한 함수
def format_data(sample):
    # 검색 결과를 문서1, 문서2... 형태로 포맷팅
    search_result = "\n-----\n".join([f"문서{idx + 1}: {result}" for idx, result in enumerate(sample["search_result"])])

    # OpenAI format으로 변환
    return {
        "messages": [
            {
                "role": "system",
                "content": system_message.format(search_result=search_result),
            },
            {
                "role": "user",
                "content": sample["question"],
            },
            {
                "role": "assistant",
                "content": sample["answer"]
            },
        ],
    }

# 분할된 데이터를 OpenAI format으로 변환
train_dataset = [format_data(dataset[i]) for i in train_data]
test_dataset = [format_data(dataset[i]) for i in test_data]

# 최종 데이터셋 크기 출력
print(f"\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Test {len(test_dataset)}개")


전체 데이터 분할 결과: Train 380개, Test 1504개


## 가공한 데이터 구조 확인

In [6]:
format_data(dataset[0])

{'messages': [{'role': 'system',
   'content': '당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.\n\n다음의 지시사항을 따르십시오.\n1. 질문과 검색 결과를 바탕으로 답변하십시오.\n2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.\n3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.\n4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.\n5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.\n6. 최대한 다수의 문서를 인용하여 답변하십시오.\n\n검색 결과:\n-----\n문서1: 열대우림 \'장마전선\'은 벵골만과 서북태평양에서 동아시아 몬순의 하위시스템으로 조성된다. \'장마전선\'의 북진 움직임은 아열대 능선이 발달한 데 영향을 받는다. 이 북쪽으로 이동하는 준정전선은 남한에서 \'장마\'라고 불리며, 주요 강수 기간을 나타낸다. 창마전선\'은 한반도를 통과하는 데 약 4~5주가 걸린다. 이러한 느린 움직임은 매년 6월말과 7월에 한반도 전체에 많은 양의 여름 강우량을 발생시킨다. 최근 들어 \'창마전선\'은 7월 말부터 8월 초까지 다양한 규모의 폭풍우와 함께 폭우가 쏟아지면서 한반도를 통과하는 데 3주도 채 걸리지 않는 등 빠르게 움직이는 경향을 보였다. \'창마\' 이후 더 극한의 날씨와 국지적인 폭우가 발생하고 있다는 뜻이다. 잠열 방출에 의해 강하게 변형된 바로크린 교란에서 비롯된 초여름의 \'창마\' 비의 역학관계는 여전히 제대로 파악되지 않고 있다. 가을 창마로 부를 수 있는 또 다른 \'창마\' 유형도 있다. 이는 물론 기상청의 공식 용어는 아니다. 그러나 최근의 기

## 분할된 데이터의 type별 분포 확인

In [6]:
print("\n학습 데이터의 type 분포:")
for type_name in set(dataset['type']):
    count = sum(1 for i in train_data if dataset[i]['type'] == type_name)
    print(f"{type_name}: {count}")

print("\n테스트 데이터의 type 분포:")
for type_name in set(dataset['type']):
    count = sum(1 for i in test_data if dataset[i]['type'] == type_name)
    print(f"{type_name}: {count}")


학습 데이터의 type 분포:
paraphrased_question: 40
mrc_question: 99
no_answer: 81
synthetic_question: 100
mrc_question_with_1_to_4_negative: 60

테스트 데이터의 type 분포:
paraphrased_question: 156
mrc_question: 392
no_answer: 323
synthetic_question: 397
mrc_question_with_1_to_4_negative: 236


## 리스트 형태에서 다시 Dataset 객체로 변경

In [7]:
print(type(train_dataset))
print(type(test_dataset))
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)
print(type(train_dataset))
print(type(test_dataset))

<class 'list'>
<class 'list'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


# 모델 로드 및 템플릿 적용

In [8]:
# 허깅페이스 모델 ID
model_id = "Qwen/Qwen3-0.6B"

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [10]:
# 템플릿 적용 확인
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system
당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
문서1: 콘코바르는 결국 죽게 되는데, 그 곡절은 이러하다. 라긴의 왕 메스 게그러의 뇌를 굳힌 겻을 울라의 코날이 전리품으로 가지고 있었는데, 코나크타의 전사 케트 막 마가크가 이를 훔쳐갔다. 그리고 케트는 무릿매로 메스 게그러의 뇌를 던져 콘코바르의 머리를 맞추었고, 메스 게그러의 뇌가 콘코바르의 머리통 깊숙히 박혀 버렸다. 이 일이 일어난 곳은 우르카르(Urchair)의 발러 아흐(Baile Ath), 곧 오늘날의 웨스트미스 주 호르셀리프라고 한다. 콘코바르의 의사들은 이 이물질을 제거할 수 없었고, 상처를 봉합만 한 뒤 왕에게 흥분하지 않으면 생명을 유지할 수 있다고 말했다. 7년이 평화롭게 흘러간 뒤 콘코바르는 그리스도가 죽었다는 소식을 듣게 되어 분노했고, 뇌가 터져 죽었다. 머리가 터진 자리에서 뿜어져나온 피의 세례를 받은 결과 그는 기독교인이 되었고 그 영혼은 천국으로 갔다. 콘코바르의 죽음에 관한 이 기록은 매우 얄팍한 기독교화가 이루어져 있는데, 한편 노르드 신화의 토르가 흐룽그니르와 싸우다 머리에 숫돌이 박힌 이야기와 유사한 점이 있다. 어쩌면 두 이야기는 하나의 기원을

# LoRA와 SFTConfig 설정

In [11]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

In [12]:
args = SFTConfig(
    output_dir="qwen3-0.6b-rag-ko",          # 저장될 디렉토리와 저장소 ID
    num_train_epochs=3,                      # 학습할 총 에포크 수
    per_device_train_batch_size=2,           # GPU당 배치 크기
    gradient_accumulation_steps=2,           # 그래디언트 누적 스텝 수
    gradient_checkpointing=True,             # 메모리 절약을 위한 체크포인팅
    optim="adamw_torch_fused",               # 최적화기
    logging_steps=10,                        # 로그 기록 주기
    save_strategy="steps",                   # 저장 전략
    save_steps=50,                           # 저장 주기
    bf16=True,                              # bfloat16 사용
    learning_rate=1e-4,                     # 학습률
    max_grad_norm=0.3,                      # 그래디언트 클리핑
    warmup_ratio=0.03,                      # 워밍업 비율
    lr_scheduler_type="constant",           # 고정 학습률
    push_to_hub=False,                      # 허브 업로드 안 함
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=[]
)

# 학습 중 전처리 함수: collate_fn

In [13]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    for example in batch:
        # messages의 각 내용에서 개행문자 제거
        clean_messages = []
        for message in example["messages"]:
            clean_message = {
                "role": message["role"],
                "content": message["content"]
            }
            clean_messages.append(clean_message)

        # 깨끗해진 메시지로 템플릿 적용
        text = tokenizer.apply_chat_template(
            clean_messages,
            tokenize=False,
            add_generation_prompt=False
        ).strip()

        # 텍스트를 토큰화
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=4096,
            padding=False,
            return_tensors=None,
        )

        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]

        # 레이블 초기화
        labels = [-100] * len(input_ids)

        # assistant 응답 부분 찾기
        im_start = "<|im_start|>"
        im_end = "<|im_end|>"
        assistant = "assistant"

        # 토큰 ID 가져오기
        im_start_tokens = tokenizer.encode(im_start, add_special_tokens=False)
        im_end_tokens = tokenizer.encode(im_end, add_special_tokens=False)
        assistant_tokens = tokenizer.encode(assistant, add_special_tokens=False)

        i = 0
        while i < len(input_ids):
            # <|im_start|>assistant 찾기
            if (i + len(im_start_tokens) <= len(input_ids) and
                input_ids[i:i+len(im_start_tokens)] == im_start_tokens):

                # assistant 토큰 찾기
                assistant_pos = i + len(im_start_tokens)
                if (assistant_pos + len(assistant_tokens) <= len(input_ids) and
                    input_ids[assistant_pos:assistant_pos+len(assistant_tokens)] == assistant_tokens):

                    # assistant 응답의 시작 위치로 이동
                    current_pos = assistant_pos + len(assistant_tokens)

                    # <|im_end|>를 찾을 때까지 레이블 설정
                    while current_pos < len(input_ids):
                        if (current_pos + len(im_end_tokens) <= len(input_ids) and
                            input_ids[current_pos:current_pos+len(im_end_tokens)] == im_end_tokens):
                            # <|im_end|> 토큰도 레이블에 포함
                            for j in range(len(im_end_tokens)):
                                labels[current_pos + j] = input_ids[current_pos + j]
                            break
                        labels[current_pos] = input_ids[current_pos]
                        current_pos += 1

                    i = current_pos

            i += 1

        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)

    # 패딩 적용
    max_length = max(len(ids) for ids in new_batch["input_ids"])

    for i in range(len(new_batch["input_ids"])):
        padding_length = max_length - len(new_batch["input_ids"][i])

        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * padding_length)
        new_batch["attention_mask"][i].extend([0] * padding_length)
        new_batch["labels"][i].extend([-100] * padding_length)

    # 텐서로 변환
    for k, v in new_batch.items():
        new_batch[k] = torch.tensor(v)

    return new_batch

# 학습

In [14]:
# collate_fn 테스트 (배치 크기 1로)
example = train_dataset[0]
batch = collate_fn([example])

print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 3762])
어텐션 마스크 형태: torch.Size([1, 3762])
레이블 형태: torch.Size([1, 3762])


In [15]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

In [16]:
# 학습 시작
trainer.train()   # 모델이 자동으로 허브와 output_dir에 저장됨

# 모델 저장
trainer.save_model()   # 최종 모델을 저장

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,0.830800
20,0.761500
30,0.714400
40,0.684400
50,0.713600
60,0.644700
70,0.706400
80,0.830800
90,0.603100
100,0.491900


# 테스트 데이터 준비

In [9]:
prompt_lst = []
label_lst = []

for prompt in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(
        prompt, tokenize=False, add_generation_prompt=False
    )
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = text.split('<|im_start|>assistant')[1]
    prompt_lst.append(input)
    label_lst.append(label)

In [10]:
print(prompt_lst[42])

<|im_start|>system
당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-----
문서1: 향교는 공자와 여러 성현께 제사를 지내고 지방민의 교육과 교화를 위해 나라에서 세운 교육기관이다.

부여향교는 언제 세웠는지 잘 알 수 없으나 조선 전기에 세운 것으로 보인다. 원래 부여읍 구교리의 서쪽 기슭에 세웠던 것을 18세기 중엽에 지금 있는 자리로 옮겼고, 여러 차례 보수를 거쳐 오늘에 이르고 있다.

건물의 전체적인 배치는 공부하는 공간인 명륜당이 앞쪽에 있고, 제사 공간인 대성전이 뒤쪽에 있어 전학후묘의 구조로 되어 있다.

명륜당은 앞면 5칸·옆면 2칸 규모로, 지붕은 옆면에서 볼 때 여덟 팔(八)자 모양인 팔작지붕 건물이다. 대성전은 앞면 3칸·옆면 3칸 규모로, 천장의 뼈대가 그대로 드러나 있는 연등천장으로 되어 있다. 안쪽에는 공자를 비롯한 중국의 성현들과 우리나라 선현들의 위패를 모시고 있다. 이외의 건물로 공부방인 수선재와 내삼문, 외삼문 등이 있다. 대성전 앞에는 다른 내삼문과는 달리 세 곳으로 나누고 협문을 두어 이를 대신하고 있다.

조선시대에는 국가로부터 토지·전적·노비 등을 지급받아 학생을 많이 가르쳤으나 갑오개혁 이후 교육 기능은 사라지고 지금은 

In [11]:
print(label_lst[42])


<think>

</think>

장성향교는 태조 3년(1394)에 처음 지어졌으나, 임진왜란으로 인해 소실되었습니다. 이후 선조 33년(1600)에 장안리로 옮겨 다시 지어졌습니다. [[ref5]]<|im_end|>



# 파인 튜닝 모델 테스트

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import pipeline

In [13]:
peft_model_id = "qwen3-0.6b-rag-ko/checkpoint-285"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

/Users/jungikim/miniconda3/envs/skn-final/lib/python3.12/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
Device set to use mps


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [14]:
eos_token = tokenizer("<|im_end|>",add_special_tokens=False)["input_ids"][0]

In [22]:
import re

def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    text = outputs[0]['generated_text'][len(prompt):].strip()
    
    # <think> ... </think> 블록 제거
    text = re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()
    return text

In [23]:
for prompt, label in zip(prompt_lst[0:1], label_lst[0:1]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    response:
1월보다 13.7% 하락한 가격으로 판매되는 신선식품은 배추입니다. 배추 한 포기(1.95㎏)의 평균 판매가는 1402원으로 10월보다 31.1% 하락했습니다. [[ref1]]
    label:

<think>

</think>

1월보다 13.7% 하락한 가격으로 판매되는 신선식품은 배추입니다. 배추 한 포기(1.95㎏)의 지난달 평균 판매가는 1402원으로, 이는 1월(1886원)보다 13.7% 내린 가격입니다 [[ref1]].<|im_end|>

--------------------------------------------------
